In [2]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(scales))
suppressPackageStartupMessages(library(gCMAP))

In [71]:
suppressPackageStartupMessages(library(parallel))

In [98]:
home_dir = '/nfs/lab/projects/selex/selex_t2d_t1d/pipeline'
well_list = readLines(paste(home_dir,'rep1/well_list.txt', sep="/" ))

In [99]:
failed = read.csv("/nfs/lab/projects/selex/selex_t2d_t1d/results/QC_correlations_reps/exclude_wells", header=T, row.names=1)
failed$well_rep = paste(failed$well, failed$rep, sep="_")


In [100]:
setwd("/nfs/lab/projects/selex/analysis/")

In [85]:
meta_analyze = function( repi, pva, sta, weigth, average_stat=TRUE) {

repi = repi[,c('snp_name', pva, sta, weigth )]
colnames(repi)[4] = "w"
repi = na.omit(repi)

### use a package to calculate z-score also when p-values are very low or 0    
repi$z_score = zScores(repi[,pva])*c(-1,1)[(repi[,sta]>0)+1]

## Combine the Z-scores using  Stouffer 1948 DOI: 10.1111/j.1420-9101.2005.00917.x
### weighted Z-scores
    
ag1 = aggregate(z_score*w~snp_name,repi, sum)
ag2 = aggregate((w^2)~snp_name,repi, sum)
czs = ag1[,2]/sqrt(ag2[,2])
    
ag1$combined_pv=pnorm(abs(czs),lower.tail=FALSE)*2


if( average_stat == TRUE ){
#### Now calculate the average of PBS and merge with the rest of the data
#avg=aggregate(repi [,sta] ~ snp_name,repi,mean) 
#avg = merge(ag1, avg, by=1)

#### Now calculate the WEIGHTED average of PBS and merge with the rest of the data
sp = split(repi, repi$snp_name)
wm = mclapply(sp, function(x) weighted.mean(x[,sta] ,x[,'w']), mc.cores = 18)
df = as.data.frame(unlist(wm))
avg = merge(ag1, df, by.x=1, by.y="row.names")   

              
colnames(avg)[3:4] = c(pva, sta)
return (avg[,c(1,3:4)])
    
   } else{
    colnames(ag1)[3] = pva
    return (ag1[,c(1,3)])
}
    
}


### 1) Combine results from the two tech reps

In [111]:
dir.create("meta_reps")

In [101]:
for (well in well_list) {
outfile = paste( 'meta_reps/' , well,  "_Enrichment_scores_reps_combined.txt", sep="")
bindi   = data.frame()    
for (vtype in c( "indels", "snvs")) {
for (rep in 1:2) {

file1 = paste( 'enrich/', well,  "_Enrichment_scores_", vtype, ".rep", rep, "dup",  sep="")
file2 = paste( '/nfs/lab/projects/selex/pipeline/', well, "/" ,vtype,  "/read_counts_rep", rep, ".txt", sep="") 

wellrep = paste(well,  "_rep", rep, sep="")
    
if(file.exists(file1)  & !(wellrep %in% failed$well_rep)){
rep1 = na.omit(read.table(file1 , header=T,  row.names=1))
rc   = read.table(file2, row.names=1)
rep1 = merge(rep1,rc, by="row.names")
    
if( (nrow(rep1)<25 & vtype=="snv") ) {rep1=NA}  ### this step is to avoid to count those with few oligos >> bad MCMC
    }else{rep1=NA}

bindi   = na.omit(rbind(bindi, rep1))
    
    }
    }   
    
if(nrow(bindi)>0){
colnames(bindi)[1] =    'snp_name'
colnames(bindi)[8] =   'read_count'
pvalues = c( 'pv_OBS','pv_PBS')  
statist = c( 'OBS' ,   'PBS' )
weigths = 'read_count'
    
for (i in 1:2){
 meta = meta_analyze (bindi,pvalues[i], statist[i], weigths )
    if(i ==1){
        bindino = meta
    }else{
    bindino = merge(bindino, meta, by=1, all.x=T)
    }
}

    ## sum of reads across reps
avg=aggregate(bindi [,"read_count"] ~ snp_name, bindi , sum) 
colnames(avg)[2]="sum_read_count" 
bindino = merge(bindino, avg, by=1, all.x=T)

write.table(bindino, outfile, sep="\t", row.names=F, quote=F)


                  }
}
    


In [112]:
head(bindino)

,snp_name,pv_OBS,OBS,pv_PBS,PBS,sum_read_count
,<I<chr>>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,T1D_1_200712429_CTAT_C,0.8132047,-0.33489500,0.8124608,-0.6704329,19
2,T1D_1_200718569_C_CTTG,0.9090884,-0.16147043,0.9147883,-0.3021014,40
3,T1D_1_200761255_A_AT,0.9594162,0.07158819,0.9624402,0.1308676,28
4,T1D_1_200844298_CT_C,0.4983580,-0.96776097,0.4981340,-1.9309116,28
5,T1D_10_33305680_A_G,0.9647241,0.07049416,0.9170403,0.2168331,68
6,T1D_10_6537425_T_C,0.8090368,-0.38178081,0.8432446,-0.4100383,32


### 2) Combine results from the same TFs

In [104]:
motif_results = read.csv("../selex_t2d_t1d/results/QC_motif_enrichment/Motif_summary.csv", row.names=1, stringsAsFactor=F)
exclude_wells = motif_results$well[ motif_results$Combined==FALSE & (motif_results$Corr<0.6 | is.na(motif_results$Corr)) ]

prot = read.table("../selex_t2d_t1d//info/TF_list.txt", header=T)
prot = subset(prot, !(protein %in% c('ORFwrong' ,'No')))
prot = subset(prot, !(well %in% exclude_wells))

In [108]:
pp = unique(prot$protein)

In [110]:
dir.create("meta_prot")

In [140]:
for (p in pp){
well_list = prot$well[prot$protein==p]
outfile   = paste( 'meta_prot/' , p,  "_Enrichment_scores_combined.txt", sep="")
bindi     = data.frame() 

for (well in well_list) {
file = paste( 'meta_reps/' , well,  "_Enrichment_scores_reps_combined.txt", sep="")
   if(file.exists(file)){
rep = read.table(file , header=T,  sep="\t")
rep$well = well    
    }else{rep=NA}
        
    bindi   = na.omit(rbind(bindi, rep))
    
    }
    
if(nrow(bindi)>0){

pvalues = c( 'pv_OBS','pv_PBS')  
statist = c( 'OBS' ,   'PBS' )
weigths = 'sum_read_count'

    
for (i in 1:2){
 meta = meta_analyze (bindi,pvalues[i], statist[i], weigths )
    if(i ==1){
        bindino = meta
    }else{
    bindino = merge(bindino, meta, by=1, all.x=T)
    }
}

aga = aggregate(well ~ snp_name, bindi, paste, collapse = ",")
agb = aggregate(well ~ snp_name, bindi, length)  
aga = merge(aga, agb, by=1)
colnames(aga)[2:3] = c("wells", "n_wells")    

avg=aggregate(bindi [,"sum_read_count"] ~ snp_name, bindi , sum)  
colnames(avg)[2]="sum_read_count" 

bindino = merge(bindino, avg, by=1, all.x=T)
bindino = merge(bindino, aga, by=1, all.x=T)


write.table(bindino, outfile, sep="\t", row.names=F, quote=F)

   } # if statement
    } # tf loops